In [1]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('static/population(seoul).csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   행정구역    350 non-null    object
 1   시도명     350 non-null    object
 2   성별      350 non-null    object
 3   연령대     350 non-null    object
 4   인구수     350 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 13.8+ KB
